<a href="https://colab.research.google.com/github/little-hoge/PythonAI/blob/main/2%E6%97%A5%E7%9B%AE/animalai/animal_ai_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install icrawler

In [ ]:
pip install flask_ngrok

In [ ]:
from flask import Flask, render_template, request
from icrawler.builtin import BingImageCrawler
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import from_numpy

import numpy as np
from PIL import Image

# ファイル参照
from google.colab import drive 
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/2日目/animalai"

classes = ["monkey","boar","crow"]
num_classes = len(classes)
image_size = 50
UPLOAD_FOLDER = './static/img'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'gif'])

#flask初期設定
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

#http://localhost:8080/にアクセスされた場合。
@app.route('/')
def index():
    return render_template('index.html')

#http://localhost:8080/にPOST送信でアクセスされた場合。
@app.route('/', methods=['POST'])
def image_judge_ai():
    
    keyword = request.form["keyword"]
    google_img_download(keyword)

    img_folder_path = os.path.join(UPLOAD_FOLDER, keyword)
    img_files = os.listdir(img_folder_path)
    
    results = []
    
    for img_file in img_files:
        results.append(animal_ai_app(img_file, keyword))

    return render_template('result.html' ,results=results ,keyword=keyword)

#画像ダウンロード
def google_img_download(keywords):

    img_dir = "static/img/" + keywords
    google_crawler = BingImageCrawler(storage={'root_dir': img_dir})
    google_crawler.crawl(keyword=keywords, max_num=10)


#画像予測判定処理
def animal_ai_app(filename, keyword):
    
    filepath = os.path.join(UPLOAD_FOLDER, keyword, filename)

    PATH = './animal_cnn.pth'
    model = Net()
    model.load_state_dict(torch.load(PATH))
    
    image = Image.open(filepath)
    image = image.convert('RGB')
    image = image.resize((image_size, image_size))
    data = np.asarray(image) / 256

    #取り込んだ画像が50×50×3の行列の為、1×50×50×3に変更
    data = np.reshape(data, (1,50,50,3))
    #1件、RGP、縦、横に変更
    data = np.transpose(data, (0, 3, 1, 2))
    #テンソルに変換
    X = from_numpy(data.astype(np.float32))

    result = torch.max(nn.functional.softmax(model(X).data ,dim=1), 1)
    
    return result_data(filename, classes[result[1]], int(result[0]*100))

#Beanクラス
class result_data():
    def __init__(self, file_name, label, probability):
        self.__file_name = file_name
        self.__label = label
        self.__probability = probability
        
    def get_file_name(self):
        return self.__file_name
    def set_file_name(self, file_name):
        self.__file_name = file_name
        
    def get_label(self):
        return self.__label
    def set_label(self, label):
        self.__label = label
        
    def get_probability(self):
        return self.__probability
    def set_file_name(self, probability):
        self.__probability = probability
    
# CNNを定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
if __name__ == '__main__':
    app.run()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/ImageJudgeAI
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e873da373090.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Feb/2021 12:21:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 12:21:44] "GET /static/css/reset.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 12:21:44] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 12:21:44] "GET /static/img/googlelogo_color.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 12:21:45] "GET /static/img/bk.jpg HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2021 12:21:46] "GET /favicon.ico HTTP/1.1" 404 -
2021-02-08 12:21:51,291 - INFO - icrawler.crawler - start crawling...
2021-02-08 12:21:51,293 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-02-08 12:21:51,296 - INFO - feeder - thread feeder-001 exit
2021-02-08 12:21:51,298 - INFO - icrawler.crawler - starting 1 parser threads...
2021-02-08 12:21:51,304 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-02-08 12:21:51,635 - INFO - parser - parsing result page https://www.bing.com/images/async?q=猿&first=0
2021-02-08 12:21:51,662 - INFO - downloader - skip d